In [1]:
from metamod.tasks import MNIST, SemanticTask
from metamod.networks import LRLinearNet
from metamod.trainers import LR_two_layer_training
from metamod.control import LinearNetEq, LRLinearNetControl
import matplotlib.pyplot as plt
import numpy as np
import copy
from tqdm import tqdm
import torch

from metamod.utils import plot_lines, plot_weight_ev, check_dir, save_var, get_date_time
from bokeh.palettes import Viridis, Category10, Category20
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.io import export_svg
output_notebook()

Loading BokehJS ...

In [2]:
run_name = "learning_rate_control"
results_path = "../results"
results_dict = {}
n_steps = 10000#15000
save_weights_every = 20
iter_control = 50
adam_lr = 0.05#0.005  # 0.005 for cost_coef 1e-4

# dataset_params = {"batch_size": 32,
#                   "h_levels": 4}

# dataset_class = SemanticTask

dataset_params = {"batch_size": 32,
                  "new_shape": (5, 5),
                  "subset": (1, 3)}

dataset_class = MNIST

model_params = {"learning_rate": 5e-3,
                "hidden_dim": 50,#30,
                "intrinsic_noise": 0.0,
                "reg_coef": 0.01,
                "W1_0": None,
                "W2_0": None}

control_params = {"control_lower_bound": -1.0,
                  "control_upper_bound": 1.0,
                  "gamma": 0.99,
                  "cost_coef": 1e-4,  # 1e-8 for aux, 1e-5 for aux_even_larger
                  "reward_convertion": 1.0,
                  "init_opt_lr": None,
                  "control_lr": adam_lr}  # 0.0005 for cost_coef 0

In [3]:
dataset = dataset_class(**dataset_params)
model_params["input_dim"] = dataset.input_dim
model_params["output_dim"] = dataset.output_dim

# model_params["W1_0"] = np.random.normal(scale=1e-4,
#                                         size=(model_params["hidden_dim"], model_params["input_dim"]))
# model_params["W2_0"] = np.random.normal(scale=1e-4,
#                                         size=(model_params["output_dim"], model_params["hidden_dim"]))

# Init neural network
model = LRLinearNet(**model_params)

/home/rodrigo/anaconda3/envs/metamod/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/rodrigo/anaconda3/envs/metamod/lib/python3.10/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/rodrigo/anaconda3/envs/metamod/lib/python3.10/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/rodrigo/anaconda3/envs/metamod/lib/python3.10/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [4]:
iters, loss, test_loss, weights_iter, weights = LR_two_layer_training(model=model, dataset=dataset, n_steps=n_steps,
                                                                      save_weights_every=save_weights_every, return_test=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 776.66it/s]


In [5]:
results_dict["iters"] = iters
results_dict["Loss_t_sim"] = loss
results_dict["weights_sim"] = weights
results_dict["weights_iters_sim"] = weights_iter
results_dict["Loss_t_sim_test"] = test_loss

# Solving equation
init_W1 = weights[0][0, ...]
init_W2 = weights[1][0, ...]

init_weights = [init_W1, init_W2]
input_corr, output_corr, input_output_corr, expected_y, expected_x = dataset.get_correlation_matrix()
input_corr_test, output_corr_test, input_output_corr_test, expected_y_test, expected_x_test = dataset.get_correlation_matrix(training=False)

time_span = np.arange(0, len(iters)) * model_params["learning_rate"]
results_dict["time_span"] = time_span

equation_params = {"in_cov": input_corr,
                   "out_cov": output_corr,
                   "in_out_cov": input_output_corr,
                   "init_weights": init_weights,
                   "in_cov_test": input_corr_test,
                   "out_cov_test": output_corr_test,
                   "in_out_cov_test": input_output_corr_test,
                   "n_steps": n_steps,
                   "reg_coef": model_params["reg_coef"],
                   "intrinsic_noise": model_params["intrinsic_noise"],
                   "learning_rate": model_params["learning_rate"],
                   "time_constant": 1.0}

solver = LinearNetEq(**equation_params)

# Initialize control
control_params = {**control_params, **copy.deepcopy(equation_params)}
control = LRLinearNetControl(**control_params)

In [6]:
W1_t, W2_t = solver.get_weights(time_span, get_numpy=True)
Loss_t = solver.get_loss_function(W1_t, W2_t, get_numpy=True)
Loss_t_test = solver.get_loss_function(W1_t, W2_t, get_numpy=True, use_test=True)

results_dict["W1_t_eq"] = W1_t
results_dict["W2_t_eq"] = W2_t
results_dict["Loss_t_eq"] = Loss_t
results_dict["Loss_t_eq_test"] = Loss_t_test

In [7]:
W1_t_control, W2_t_control = control.get_weights(time_span, get_numpy=True)
Loss_t_control = control.get_loss_function(W1_t_control, W2_t_control, get_numpy=True)
Loss_t_control_test = control.get_loss_function(W1_t, W2_t, get_numpy=True, use_test=True)

results_dict["W1_t_control_init"] = W1_t_control
results_dict["W2_t_control_init"] = W2_t_control
results_dict["Loss_t_control_init"] = Loss_t_control
results_dict["Loss_t_control_init_test"] = Loss_t_control_test
results_dict["control_signal_init"] = control.opt_lr.detach().cpu().numpy()

In [8]:
losses = (loss, Loss_t, Loss_t_control, Loss_t_test, Loss_t_control_test)
colors = (Category10[10][0], Category10[10][0], Category10[10][1], Category10[10][2], Category10[10][3])
legends = ("Real Non-linear", "Approximation", "Init Control", "eq test", "control test")
alphas = (0.3, 1, 1, 1, 1)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

In [9]:
control_params["iters_control"] = iter_control
cumulated_reward = []
mean_grad = []

for i in tqdm(range(iter_control)):
    R, grad = control.train_step(get_numpy=True)
    cumulated_reward.append(R)
    mean_grad.append(np.mean(grad**2))
cumulated_reward = np.array(cumulated_reward).astype(float)
results_dict["cumulated_reward_opt"] = cumulated_reward

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [06:12<00:00,  7.45s/it]


In [10]:
opt = plot_lines(np.arange(iter_control), (cumulated_reward,), x_axis_label="gradient steps on control", y_axis_label="Cumulated reward")
show(opt)

In [11]:
W1_t_opt, W2_t_opt = control.get_weights(time_span, get_numpy=True)
Loss_t_opt = control.get_loss_function(W1_t_opt, W2_t_opt, get_numpy=True)
Loss_t_opt_test = control.get_loss_function(W1_t_opt, W2_t_opt, get_numpy=True, use_test=True)

results_dict["W1_t_control_opt"] = W1_t_opt
results_dict["W2_t_control_opt"] = W2_t_opt
results_dict["Loss_t_control_opt"] = Loss_t_opt

In [19]:
losses = (loss, Loss_t, Loss_t_opt, Loss_t_opt_test)
colors = (Category10[10][0], Category10[10][0], Category10[10][1], Category10[10][2])
legends = ("Real Non-linear", "Approximation", "Approximated Optimized Control", "test")
alphas = (0.3, 1, 1, 1)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

In [20]:
opt_lr = (control.opt_lr.detach() + torch.ones(control.opt_lr.shape, dtype=model.dtype, device=model.device))*model_params["learning_rate"]
control_signal = opt_lr
W1_0, W2_0 = control_params["init_weights"]
results_dict["control_signal"] = opt_lr

In [21]:
opt_lr

tensor([0.0061, 0.0061, 0.0061,  ..., 0.0050, 0.0050, 0.0050], device='cuda:0')

In [22]:
reset_model_params = model_params.copy()
reset_model_params["W1_0"] = W1_0
reset_model_params["W2_0"] = W2_0

reset_model = LRLinearNet(**reset_model_params)

In [23]:
iters, loss_OPT, loss_OPT_test, weights_iter_OPT, weights_OPT  = LR_two_layer_training(model=reset_model, dataset=dataset, n_steps=n_steps,
                                                                                       save_weights_every=save_weights_every,
                                                                                       opt_lr=opt_lr, return_test=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 894.62it/s]


In [25]:
losses = (loss, Loss_t, Loss_t_opt, loss_OPT, loss_OPT_test, Loss_t_opt_test)
colors = (Category10[10][0], Category10[10][0], Category10[10][1], Category10[10][1], Category10[10][2], Category10[10][2])
legends = ("Real Non-linear", "Approximation", "Approximated Optimized Control", "Optimized Non-linear", "opt_test_sim", "opt_test")
alphas = (0.3, 1, 1, 0.3, 0.3, 1)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)